$$ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 + b $$
$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$
 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

# Naive data representation

In [3]:
# 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()   # 기존의 기울기(gradient) 초기화
    cost.backward()         # 비용 함수 미분 (기울기 계산)_역전파과정
    optimizer.step()        # 미분된 값을 사용해 가중치, 편향 업데이트_경사 하강법을 사용해 w1, w2, w3, b를 업데이트

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} | w1: {:.3f} | w2: {:.3f} | w3: {:.3f} | b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))
        # .item(): 텐서 값을 일반 숫자로 변환

Epoch    0/1000 | w1: 0.294 | w2: 0.294 | w3: 0.297 | b: 0.003 Cost: 29661.800781
Epoch  100/1000 | w1: 0.674 | w2: 0.661 | w3: 0.676 | b: 0.008 Cost: 1.563634
Epoch  200/1000 | w1: 0.679 | w2: 0.655 | w3: 0.677 | b: 0.008 Cost: 1.497607
Epoch  300/1000 | w1: 0.684 | w2: 0.649 | w3: 0.677 | b: 0.008 Cost: 1.435026
Epoch  400/1000 | w1: 0.689 | w2: 0.643 | w3: 0.678 | b: 0.008 Cost: 1.375730
Epoch  500/1000 | w1: 0.694 | w2: 0.638 | w3: 0.678 | b: 0.009 Cost: 1.319511
Epoch  600/1000 | w1: 0.699 | w2: 0.633 | w3: 0.679 | b: 0.009 Cost: 1.266222
Epoch  700/1000 | w1: 0.704 | w2: 0.627 | w3: 0.679 | b: 0.009 Cost: 1.215696
Epoch  800/1000 | w1: 0.709 | w2: 0.622 | w3: 0.679 | b: 0.009 Cost: 1.167818
Epoch  900/1000 | w1: 0.713 | w2: 0.617 | w3: 0.680 | b: 0.009 Cost: 1.122429
Epoch 1000/1000 | w1: 0.718 | w2: 0.613 | w3: 0.680 | b: 0.009 Cost: 1.079378


## Matrix data representation

$$
\begin{pmatrix}
x_1 & x_2 & x_3
\end{pmatrix}
\cdot
\begin{pmatrix}
w_1 \\
w_2 \\
w_3 \\
\end{pmatrix}
=
\begin{pmatrix}
x_1w_1 + x_2w_2 + x_3w_3
\end{pmatrix}
$$

$$ H(X) = XW $$

In [5]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [6]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = x_train @ W + b    # .matmul() or .mm()도 가능, (5, 3) 크기

    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712891
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

In [7]:
# 1000번 훈련시,
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = x_train @ W + b    # .matmul() or .mm() 도 가능

    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))
    
# squeeze() 로 (불필요한) 1차원(축)을 제거하여 깔끔히 만들어 줌
# detach() 로 그래디언트 계산 비활성화하여 텐서를 numpy 배열처럼 값만 남긴 상태로 변환
# 즉 2개를 같이 쓰는 방법이, 학습 과정에서 결과를 출력할 때 필요하다.

Epoch    0/1000 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch  100/1000 hypothesis: tensor([152.7691, 183.6985, 180.9591, 197.0627, 140.1336]) Cost: 1.563634
Epoch  200/1000 hypothesis: tensor([152.7273, 183.7273, 180.9465, 197.0517, 140.1731]) Cost: 1.497603
Epoch  300/1000 hypothesis: tensor([152.6866, 183.7554, 180.9343, 197.0409, 140.2116]) Cost: 1.435026
Epoch  400/1000 hypothesis: tensor([152.6470, 183.7827, 180.9224, 197.0304, 140.2491]) Cost: 1.375730
Epoch  500/1000 hypothesis: tensor([152.6085, 183.8093, 180.9108, 197.0202, 140.2856]) Cost: 1.319503
Epoch  600/1000 hypothesis: tensor([152.5711, 183.8352, 180.8996, 197.0101, 140.3211]) Cost: 1.266215
Epoch  700/1000 hypothesis: tensor([152.5346, 183.8604, 180.8887, 197.0003, 140.3558]) Cost: 1.215693
Epoch  800/1000 hypothesis: tensor([152.4992, 183.8849, 180.8781, 196.9908, 140.3895]) Cost: 1.167821
Epoch  900/1000 hypothesis: tensor([152.4647, 183.9087, 180.8677, 196.9814, 140.4223]) Cost: 1.122419
Epoch 

High-level implementation w/ `nn.Module`

이전 시간에 진행했던 LinearRegressionModel 클래스는 아래와 같은데,

In [8]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

multivariable 상황으로 맞추려면,

nn.Linear의 input dimension을 1에서 3으로 바꾸면 된다:

In [9]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [10]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    prediction = model(x_train)
    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 31667.599609
Epoch    1/20 Cost: 9926.265625
Epoch    2/20 Cost: 3111.513916
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042496
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762
Epoch   20/20 Cost: 0.227732


In [ ]:
# 100번 학습할 경우,
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    # H(x) 계산_prediction이라 썼는데, 위의 hypothesis와 같다
    prediction = model(x_train)
    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 24821.373047
Epoch   10/100 Cost: 1.098634
Epoch   20/100 Cost: 0.868905
Epoch   30/100 Cost: 0.866380
Epoch   40/100 Cost: 0.863868
Epoch   50/100 Cost: 0.861380
Epoch   60/100 Cost: 0.858916
Epoch   70/100 Cost: 0.856444
Epoch   80/100 Cost: 0.854000
Epoch   90/100 Cost: 0.851573
Epoch  100/100 Cost: 0.849148
